In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import pickle

metadata = pickle.load(open('metadata.pkl', "rb"))

In [4]:
import numpy as np
from math import ceil
import tensorflow as tf
from malaya_speech.train.model import fastspeech, transformervc
import malaya_speech






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [7]:
sbmt_i = metadata[0]
x_org = sbmt_i[2]
c = np.random.normal(size = (2, 512)).astype(np.float32)

In [8]:
mel = tf.placeholder(tf.float32, (None, None, 80))
ori_vector = tf.placeholder(tf.float32, (None, 512))
target_vector = tf.placeholder(tf.float32, (None, 512))
mel_lengths = tf.placeholder(tf.int32, (None))

In [9]:
dim_neck = 32
dim_speaker = 512
dim_input = 80
encoder_config = malaya_speech.config.transformer_config.copy()
decoder_config = malaya_speech.config.transformer_config.copy()
encoder_config['hidden_size'] = dim_speaker + dim_input
decoder_config['hidden_size'] = dim_speaker + dim_neck
encoder_config['activation'] = 'mish'
decoder_config['activation'] = 'mish'

In [10]:
config = malaya_speech.config.fastspeech_config
config = fastspeech.Config(vocab_size = 1, **config)

In [11]:
model = transformervc.Model(dim_neck, encoder_config, decoder_config, config, dim_input = dim_input,
                           dim_speaker = dim_speaker, skip = 6)

In [12]:
encoder_outputs, mel_before, mel_after, codes = model(mel, ori_vector, target_vector, mel_lengths)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [13]:
codes_ = model.call_second(mel_after, ori_vector, mel_lengths)

In [14]:
encoder_outputs, mel_before, mel_after, codes, codes_

(<tf.Tensor 'fastvc/GatherV2:0' shape=(?, ?, 544) dtype=float32>,
 <tf.Tensor 'fastvc/mel_before/BiasAdd:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'fastvc/add:0' shape=(?, ?, 80) dtype=float32>,
 <tf.Tensor 'fastvc/Encoder/Transformer/encode/encoder_dense/BiasAdd:0' shape=(?, ?, 32) dtype=float32>,
 <tf.Tensor 'Encoder/Transformer/encode/encoder_dense/BiasAdd:0' shape=(?, ?, 32) dtype=float32>)

In [15]:
loss_f = tf.losses.mean_squared_error
max_length = tf.cast(tf.reduce_max(mel_lengths), tf.int32)
mask = tf.sequence_mask(
    lengths = mel_lengths, maxlen = max_length, dtype = tf.float32
)
mask = tf.expand_dims(mask, axis = -1)
mask

<tf.Tensor 'ExpandDims:0' shape=<unknown> dtype=float32>

In [16]:
from functools import partial
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

mse_mel = partial(
    loss_f,
    weights = mask
)

In [17]:
mel_loss_before = calculate_3d_loss(mel, mel_before, mse_mel)
mel_loss_before

<tf.Tensor 'mean_squared_error/value:0' shape=() dtype=float32>

In [18]:
mel_loss_after = calculate_3d_loss(mel, mel_after, mse_mel)
mel_loss_after

<tf.Tensor 'mean_squared_error_1/value:0' shape=() dtype=float32>

In [19]:
g_loss_cd = tf.losses.absolute_difference(codes, codes_)

In [20]:
loss = mel_loss_before + mel_loss_after + g_loss_cd

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [22]:
o = sess.run([encoder_outputs, mel_before, mel_after], feed_dict = {mel: [x_org] * 2,
                                                                    ori_vector: c,
                                                                    target_vector: c,
                                                                    mel_lengths: [len(x_org)] * 2})

In [23]:
o[0].shape, o[1].shape, o[2].shape

((2, 90, 544), (2, 90, 80), (2, 90, 80))

In [24]:
o = sess.run([mel_loss_before, mel_loss_after, g_loss_cd], feed_dict = {mel: [x_org] * 2,
                                                                    ori_vector: c,
                                                                    target_vector: c,
                                                                    mel_lengths: [len(x_org)] * 2})

In [25]:
o

[2.4486876, 3.4944894, 0.8022246]

In [26]:
tf.trainable_variables()

[<tf.Variable 'Transformer/encoder_stack/layer_0/self_attention/layer_normalization/layer_norm_scale:0' shape=(592,) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/self_attention/layer_normalization/layer_norm_bias:0' shape=(592,) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/self_attention/self_attention/q/kernel:0' shape=(592, 592) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/self_attention/self_attention/k/kernel:0' shape=(592, 592) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/self_attention/self_attention/v/kernel:0' shape=(592, 592) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/self_attention/self_attention/output_transform/kernel:0' shape=(592, 592) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/ffn/layer_normalization/layer_norm_scale:0' shape=(592,) dtype=float32_ref>,
 <tf.Variable 'Transformer/encoder_stack/layer_0/ffn/layer_normalization/l

In [27]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, "test/model.ckpt")

'test/model.ckpt'

In [28]:
!ls -lh test
!rm -rf test

total 421040
-rw-r--r--  1 huseinzolkepli  staff    77B May 19 23:18 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   197M May 19 23:18 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    10K May 19 23:18 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   3.3M May 19 23:18 model.ckpt.meta
